# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
# YOUR CODE HERE
# Creating a folder to store Python script

import os
os.makedirs("going_modular", exist_ok=True)

## 1.1 Getting data and turn the code into python script(get_data.py)

In [2]:
## Script mode
%%writefile going_modular/get_data.py
import os
import zipfile
import requests
from pathlib import Path

# Setup the data path and image path
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
  print(f"{image_path} directory exists.")
else:
  print(f"Did not find {image_path} directory, creating one...")
  image_path.mkdir(parents=True, exist_ok=True)

  # Download pizza, steak, sushi data
  with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

  # Unzip the datasets after finish downloading
  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

  # remove zipfile
  os.remove(data_path / "pizza_steak_sushi.zip")

Writing going_modular/get_data.py


In [3]:
 # Example running of get_data.py
!python going_modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

---

## 2.1 Create Datasets and DataLoaders in script mode (data_setup.py)

In [4]:

%%writefile going_modular/data_setup.py

""""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""

import os

# Import all nessesary library
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transforms: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """
  Create training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = create
      dataloaders(train_dir=path/to/train_dir,
                  test_dir=path/to/test_dir,
                  transform=some_transform,
                  batch_size=32,
                  num_workers=4)
  """

  # Use ImageFolder to create Dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transforms, target_transform=None)
  test_data = datasets.ImageFolder(test_dir, transform=transforms, target_transform=None)

  # Get class name
  class_names = train_data.classes

  # Turn image into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names


Writing going_modular/data_setup.py


## 2.2 Creating model(TinyVGG) in script mode (model_builder.py)

In [5]:
%%writefile going_modular/model_builder.py
"""
Contains all necessary code for building a TinyVGG model.
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
  """
  Create the TinyVGG architecture.

  This is the replicated architecture from the CNN explainer website in Pytorch
  Original: https://poloclub.github.io/cnn-explainer/

  Arg:
    input_shape: An integer indicating number of input channels.
    hidden_shape: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self,
               input_shape: int,
               hidden_shape: int,
               output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_shape,
                  kernel_size=3, # How big is the squarre that going over image
                  stride=1, # defailt value
                  padding=0), # optinal
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_shape, # Changed input_shape to hidden_shape
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_shape,
                  out_channels=hidden_shape,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.classifier = nn.Sequential( # Changed classiferr to classifier
        nn.Flatten(),
        nn.Linear(in_features=hidden_shape*13*13,
                  out_features=output_shape)
    )
  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

Writing going_modular/model_builder.py


## 2.3 Creating `train_step()` and `test_step()` functions and `train() to combind them (engine.py)

This will act as an engine for training and testing.

In [6]:
%%writefile going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
from typing import Tuple, Dict, List
import torch
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloaders: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """
  Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).
  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").
  """
  # Put the model into train mode
  model.train()

  # Setup train and train acc values
  train_loss, train_acc = 0, 0

  #Loop through the batches of data
  for batch, (X, y) in enumerate(dataloaders):
    # Send data through the target device
    X, y = X.to(device), y.to(device)

    # 1. Forward pass
    y_pred = model(X)

    # 2. Caculated the loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # 3. Optimizer the zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # caculated and accumualated acc metrics across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get avg loss and acc per batch
  train_loss = train_loss / len(dataloaders)
  train_acc = train_acc / len(dataloaders)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloaders: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """
  Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.
  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").
  """
  # Put model into eval mode
  model.eval()
  # Setup test loss and test acc
  test_loss, test_acc = 0, 0
  # turn on inference mode
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloaders):
      # Send data through the target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      test_pred_logits = model(X)

      # 2. Caculated the loss
      loss =  loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # Caculate and accumulate acc metrics
      test_pred_labels = test_pred_logits.argmax(dim=1)
      test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get avg loss and acc per batches
  test_loss = test_loss / len(dataloaders)
  test_acc = test_acc / len(dataloaders)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:

  # Create an empty tray of result
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # Loop through the batches of data for trarining and testings with
  # number of epochs

  for epochs in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                        dataloaders=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloaders=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # Print out what happening
    print(
        f"Epochs: {epochs+1}",
        f"train_loss: {train_loss:.4f} |",
        f"train_acc: {train_acc:.4f} |",
        f"test_loss: {test_loss:.4f} |",
        f"test_acc: {test_acc:.4f}"
    )

    # Update the results dict
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  return results




Writing going_modular/engine.py


## 2.4 Create a function to save the model in script mode

This function is to save our model to the directory

In [7]:
%%writefile going_modular/utils.py
from pathlib import Path
import torch
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name:str):
  """
  Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model.
  """
  # Create a target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create a model save Path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_ddict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


## 2.5 Train, evaluate and save the model in script mode

Use Python's argparse module to be able to send the train.py custom
hyperparameter values for training procedures.

* Add an argument flag for using a different:
* Training/testing directory
* Learning rate
* Batch size
* Number of epochs to train for
* Number of hidden units in the TinyVGG model
* Keep the default values for each of the above arguments as what they already are (as in notebook 05).

In [8]:
%%writefile going_modular/train.py
"""
Trains a Pytorch model
"""
import os
import torch
from torchvision import transforms
import argparse
from argparse import ArgumentParser
import sys # Import sys
from pathlib import Path # Import Path

# Add the parent directory of the script's location to the Python path
sys.path.append(str(Path(__file__).parent.parent))

import going_modular.data_setup as data_setup
import going_modular.engine as engine
import going_modular.model_builder as model_builder
import going_modular.utils as utils


parser = argparse.ArgumentParser(description="Get some hyperparameters.")
# Setup some args for parsers
parser.add_argument('--NUM_EPOCHS',
                    default=10,
                    type=int,
                    help="Number of time want to train")
parser.add_argument('--BATCH_SIZE',
                    default=32,
                    type=int,
                    help="Number of samples per batch")
parser.add_argument('--HIDDEN_SHAPE',
                    default=10,
                    type=int,
                    help="Number of hidden units in the TinyVGG model")
parser.add_argument('--LEARNING_RATE',
                    default=0.001,
                    type=float,
                    help="Learning rate to use for model")
parser.add_argument('--train_dir',
                    default="data/pizza_steak_sushi/train",
                    type= str,
                    help="Directory file path to training data in standard image classification format")
parser.add_argument('--test_dir',
                    default="data/pizza_steak_sushi/test",
                    type=str,
                    help="Directory file path to testing data in standard image classification format")
parser.add_argument('--device',
                    default="cuda" if torch.cuda.is_available() else "cpu",
                    type=str,
                    help="Device default is cuda else it train on CPU")


# After finish setting up hyperarameters, get arguments from the parser
args = parser.parse_args()

# Setup some hyperparameters
NUM_EPOCHS = args.NUM_EPOCHS
BATCH_SIZE = args.BATCH_SIZE
HIDDEN_SHAPE = args.HIDDEN_SHAPE
LEARNING_RATE = args.LEARNING_RATE
print(f"[INFO] Model was training on {NUM_EPOCHS} epochs, with {BATCH_SIZE} size, {HIDDEN_SHAPE} hidden units and {LEARNING_RATE} learning rate")

train_dir = args.train_dir
test_dir = args.test_dir
print(f"[INFO] the Datasetts was taken from{train_dir} and {test_dir}")

device = args.device

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transforms=data_transform,
    batch_size=BATCH_SIZE
)

# Create model from the help of model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_shape=HIDDEN_SHAPE,
    output_shape=len(class_names)
).to(device)

# Setup loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)
# Create train and test with the help of engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Create a save model with the help of utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [9]:
# Example running of train.py
!python going_modular/train.py --NUM_EPOCHS 5 --BATCH_SIZE 128 --HIDDEN_SHAPE 128 --LEARNING_RATE 0.0003

[INFO] Model was training on 5 epochs, with 128 size, 128 hidden units and 0.0003 learning rate
[INFO] the Datasetts was taken fromdata/pizza_steak_sushi/train and data/pizza_steak_sushi/test
  0% 0/5 [00:00<?, ?it/s]Epochs: 1 train_loss: 1.1019 | train_acc: 0.3187 | test_loss: 1.0972 | test_acc: 0.3333
 20% 1/5 [00:02<00:10,  2.59s/it]Epochs: 2 train_loss: 1.0951 | train_acc: 0.3521 | test_loss: 1.0919 | test_acc: 0.4133
 40% 2/5 [00:03<00:05,  1.74s/it]Epochs: 3 train_loss: 1.0797 | train_acc: 0.4909 | test_loss: 1.0762 | test_acc: 0.3733
 60% 3/5 [00:04<00:02,  1.45s/it]Epochs: 4 train_loss: 1.0575 | train_acc: 0.4301 | test_loss: 1.0510 | test_acc: 0.3867
 80% 4/5 [00:05<00:01,  1.31s/it]Epochs: 5 train_loss: 0.9977 | train_acc: 0.5323 | test_loss: 1.0207 | test_acc: 0.4267
100% 5/5 [00:07<00:00,  1.40s/it]
[INFO] Saving model to: models/05_going_modular_script_mode_tinyvgg_model.pth


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [59]:
%%writefile going_modular/predict.py
import torch
import torchvision
import argparse
import model_builder
from torchvision import transforms
import matplotlib.pyplot as plt

parser = argparse.ArgumentParser(description="Get some hyperparameters.")
parser.add_argument('--image',
                    help="Target image filepath to predict on")
parser.add_argument('--model_path',
                    default="models/05_going_modular_script_mode_tinyvgg_model.pth",
                    type=str,
                    help="Target model to use for prediction filepath")

args = parser.parse_args()

# Setup class name
class_names = ["pizza", "steak", "sushi"]

# Setup device agconostic
device = "cuda" if torch.cuda.is_available() else "cpu"

# Ge the image patht
IMG_PATH = args.image
print(f"[INFO] Predicting on {IMG_PATH}")

# Function to load the model
def load_model(file_path=args.model_path):
  model = model_builder.TinyVGG(
    input_shape=3,
    hidden_shape=128,
    output_shape=len(class_names)).to(device)
  print(f"[INFO] Loading in model from: {file_path}")
  # Load in he saved model sate dicitonary from file
  model.load_state_dict(torch.load(file_path))
  return model

# function to load the image and use the model to predict
def predict_img(img_path=IMG_PATH,
                file_path=args.model_path):

  # Load in the model
  model = load_model(file_path)

  # 1. Load in image and convert the tensor value to float32
  img = torchvision.io.read_image(str(IMG_PATH)).type(torch.float32)

  # 2.Divide the image pixel by 255 to get between [0,1]
  img = img / 255.0

  # 3.Transform the img
  transform = torchvision.transforms.Resize((64, 64))
  img = transform(img)


  # 4. Make sure the model is in targe device
  model.to(device)

  # 5. Turn on model eval mode and inference mode to preict image
  model.eval()
  with torch.inference_mode():
    # Put image o target device
    img = img.to(device)

    # 5.1 Get preds logits
    img_pred_logits = model(img.unsqueeze(dim=0))

    # 5.2 Convert pred logit -> preds probs
    img_pred_probs = torch.softmax(img_pred_logits, dim=1)

    # 5.3 Convert pred probs -> preds labels
    img_pred_labels = torch.argmax(img_pred_probs, dim=1)
    pred_class = class_names[img_pred_labels]

  print(f"[INFO] Pred class: {pred_class}, Pred Probs: {img_pred_probs.max():.3f}")

if __name__ == "__main__":
  predict_img()

Overwriting going_modular/predict.py


In [60]:
# Example running of predict.py
!python going_modular/predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

[INFO] Predicting on data/pizza_steak_sushi/test/sushi/175783.jpg
[INFO] Loading in model from: models/05_going_modular_script_mode_tinyvgg_model.pth
[INFO] Pred class: sushi, Pred Probs: 0.402
